In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
os.symlink('/content/mnt/My Drive/Colab Notebooks', '/content/notebooks')
sys.path.insert(0, '/content/notebooks')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


FileExistsError: ignored

In [ ]:
from zipfile import ZipFile
%cd /content/notebooks
!unzip bdd100k_images.zip
!unzip bdd100k_labels_release.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: bdd100k/images/10k/train/3d187d59-48023a43.jpg  
 extracting: bdd100k/images/10k/train/3d1a6305-3980e3bc.jpg  
 extracting: bdd100k/images/10k/train/3d2c1ef5-fdc7bea5.jpg  
 extracting: bdd100k/images/10k/train/3d3117f0-9365aac0.jpg  
 extracting: bdd100k/images/10k/train/3d317eb1-4a9f2f5c.jpg  
 extracting: bdd100k/images/10k/train/3d31c23f-bb0e2813.jpg  
 extracting: bdd100k/images/10k/train/3d31d9a7-2fa8d37a.jpg  
 extracting: bdd100k/images/10k/train/3d3a3638-268c8d7f.jpg  
 extracting: bdd100k/images/10k/train/3d3a3638-d2cf7766.jpg  
 extracting: bdd100k/images/10k/train/3d3f2fa6-666e1cf1.jpg  
 extracting: bdd100k/images/10k/train/3d411c3d-1324dab3.jpg  
 extracting: bdd100k/images/10k/train/3d411c3d-a9202cf2.jpg  
 extracting: bdd100k/images/10k/train/3d507efc-2d6c0219.jpg  
 extracting: bdd100k/images/10k/train/3d50b2fd-031cce4f.jpg  
 extracting: bdd100k/images/10k/train/3d5169ab-8ec93ab4.jpg  
 extracting: bdd100k/images/10k/tr

In [ ]:
import os, json

bdd_labels = ["bike", "bus", "car", "motor", "person", "rider", "traffic light", "traffic sign", "train", "truck"]
bdd_labels_reverse = {}
for i, label in enumerate(bdd_labels):
    bdd_labels_reverse[label] = i
    
def convert_bdd100k_to_yolo_label(json_path, output_path):
    with open(json_path) as json_file:
        for element in json.load(json_file):
            file_id = os.path.splitext(element['name'])[0]
            with open(os.path.join(output_path, file_id + '.txt'), 'w') as txt_file:
                for label in element['labels']:
                    if not label['category'] in bdd_labels: continue
                    label_id = bdd_labels_reverse[label['category']]
                    x1, y1, x2, y2 = float(label['box2d']['x1']) / 1280, float(label['box2d']['y1']) / 720, \
                                     float(label['box2d']['x2']) / 1280, float(label['box2d']['y2']) / 720
                    x, y, w, h = (x1 + x2) / 2, (y1 + y2) / 2, abs(x1 - x2), abs(y1 - y2)
                    txt_file.write(f'{label_id} {x} {y} {w} {h}\n')

%cd /content/notebooks
os.makedirs('bdd100k/labels/100k/train', exist_ok=True)
os.makedirs('bdd100k/labels/100k/val', exist_ok=True)
convert_bdd100k_to_yolo_label('bdd100k/labels/bdd100k_labels_images_train.json', 'bdd100k/labels/100k/train')
convert_bdd100k_to_yolo_label('bdd100k/labels/bdd100k_labels_images_val.json', 'bdd100k/labels/100k/val')

/content/mnt/My Drive/Colab Notebooks


In [ ]:
%cd /content/notebooks
!git clone https://github.com/ultralytics/yolov5  # clone repo

cp bdd100k.yaml yolov5/data
cp -r yolov5s-bdd.yaml yolov5/models/yolov5s.yaml
cp -r yolov5m-bdd.yaml yolov5/models/yolov5m.yaml

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


In [2]:
%cd /content/notebooks
%pip install -qr yolov5/requirements.txt  # install dependencies

/content/mnt/My Drive/Colab Notebooks
     |████████████████████████████████| 276kB 6.7MB/s 


In [ ]:
!cd yolov5 && python train.py --img 1280 --batch 4 --epochs 10 --data bdd100k.yaml --cfg yolov5m.yaml --weights yolov5m.pt

Using torch 1.7.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=False, cfg='./models/yolov5m.yaml', data='./data/bdd100k.yaml', device='', epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[1280, 1280], local_rank=-1, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/exp2', single_cls=False, sync_bn=False, total_batch_size=4, weights='yolov5m.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2020-11-17 17:42:45.617225: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1,

In [6]:
!cd yolov5 && python test.py --data bdd100k.yaml --weights yolov5s.pt --img 1280

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='./data/bdd100k.yaml', device='', exist_ok=False, img_size=1280, iou_thres=0.65, name='exp', project='runs/test', save_conf=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['yolov5s.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla P4, 7611MB)

Fusing layers... 
Model Summary: 232 layers, 7459581 parameters, 0 gradients
Scanning labels ../bdd100k/labels/100k/val.cache (10000 found, 0 missing, 0 empty, 0 duplicate, for 10000 images): 10000it [00:00, 14219.85it/s]
               Class      Images     Targets           P           R      mAP@.5  mAP@.5:.95: 100% 313/313 [30:54<00:00,  5.93s/it]
                 all       1e+04    1.86e+05      0.0471       0.132      0.0839       0.041
Speed: 53.8/4.0/57.8 ms inference/NMS/total per 1280x1280 image at batch-size 32
Results saved to runs/test/exp2
